In [1]:
# Импортируем библиотеки и смотрим версию
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
print( f"pandas {pd.__version__}, plotly {plotly.__version__}, np {np.__version__}" )
# pandas 2.3.0, plotly 6.2.0, np 2.2.6

pandas 2.3.0, plotly 6.2.0, np 2.2.6


In [2]:
# Читаем перчичный файл
churn_data = pd.read_csv('data/churn.csv')
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Копируем датафрейм и смотрим структуру
churn_df = churn_data.copy()
churn_df = churn_df.drop('RowNumber', axis = 1)
#churn_df.describe()
churn_df.info()
#churn_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


9.1. Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [7]:
tmp_df = churn_df[['Exited']].copy()

# Вычислим долю ушедших (в %), округлим до целых
df = (tmp_df.mean() * 100).round(0).astype( int )

# Готовим графику: имеем статистические данные, используем пончиковую круговую диаграмму
fig = px.pie( 
    values = [100 - df.values[0], df.values[0]], # Процентное соотношение
    title = 'Соотношение ушедших и лояльных клиентов', # Заголовок отчета
    hole = 0.7,  # Настроим так ширину бублика
    names = ["Лояльные", "Ушедшие" ], # Настроим Подписи
    width = 450, # Ширина
    height = 450 # Высота
)
fig.update_traces( textposition = 'inside', textinfo = 'percent+label', showlegend = False )
fig.show() # Отображаем

Видим, что лояльных клиентов вчетверо больше, чем успешных, банк успешный

9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [13]:
# Выбираем подмножество данных согласно условиям
tmp_df = churn_df[churn_df["Balance"] > 2500][["Balance"]].copy()

# Готовим графику: имеем распределение первичных данных, используем коробчатую диаграмму
fig = px.box(
    data_frame = tmp_df,  
    x = "Balance", # По оси x отпечатаются балансы 
    title = 'Распределение по балансам клиентов с балансом > 2500', # Заголовок отчета
    width = 600,  # Ширина графика
    height = 400, # Высота графика
)
fig.update_layout(
    bargap = 0.1, # Ширина вертикальной разделительной полосы
    xaxis_title = "Баланс клиента, $", # Подпись оси x
    yaxis_title = "Вск клиенты", # Подпись оси y
    legend = dict( orientation = "h", x = 0, y = 150, title_font_family = "Sitka Small" ) # Сместим легенду слева вверху
)
fig.update_traces( quartilemethod = "exclusive") # Форма диаграммы
fig.show() # Отображаем

Из графика видим, клиенты распределены от 41 до 199 тыс $. Медианный баланс 120 тыс $, максимальное распределение клиентов в отрезке от 100 (25% квантиль) до 140 (75% квантиль) тыс $. Выбросы слева и справа примерно одинаковой ширины


9.3. Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [14]:
# Выбираем нужные признаки
tmp_df = churn_df[["Balance", 'Exited','CustomerId']].copy()

# Группируем и подписываем данные
# Долю оттока к общему количеству выводить не будем, дадим вместо этого общее количество.
df = tmp_df.pivot_table(
    index = 'Balance',
    columns = 'Exited',
    values = 'CustomerId',
    aggfunc = 'count'
)
df = df.rename( columns = {1: "Ушедшие", 0: "Лояльные"})

# Готовим графику: имеем распределение первичных данных, используем гистограммы
fig = px.histogram(
    data_frame = df,
    x = df.index,   # По оси x отпечатаются балансы клиентов
    y = df.columns, # По оси y отпечатается количество лояльных и ушедших клиентов
    nbins = 50, # Число интервалов 
    text_auto = True, # Отображать надписи на колонках гистограммы
    title = 'Распределение по балансам', #заголовок
    width = 1000, # Ширина графика
    height = 500, # Высота графика
    labels = {'Exited': 'Статус клиента: '} # Подпись на легенде
)
fig.update_layout(
    bargap = 0.1, # Ширина вертикальной разделительной вертикальной полосы
    xaxis_title = "Баланс клиента, $", # Подпись оси x
    yaxis_title = "Количество", # Подпись оси y
    legend = dict( orientation = "h", x = 0, y = 150, title_font_family = "Sitka Small" ) # Сместим легенду воева вверх
)
fig.show() # Отображаем

Из графика видим, что 
1) Максимальное количество клиентов (как лояльных, так и ушедших) находится в диапазоне 90-150 тыс долларов.
2) Доля ушедших в диапазоне с балансом 90-150 тыс немного выше, чем вне диапазона
3) Пик в начале графика - это новые клиенты, которые решили попрововать открыть счет в банке, положили некую тестовую сумму. Большинство из них осталось и часть ушла.
4) По краям графика колебания оконо нулевой отметки. Возможно это разовые операции, клиенты не собирались долгосрочно открывать счет

9.4. Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [175]:
# Выбираем нужные признаки
tmp_df = churn_df[['Age','Exited','CustomerId']].copy()

# Группируем и подписываем данные
df = tmp_df.groupby( 'Age', as_index = False )['Exited'].mean().round(2) 
df["Отток"] = (df["Exited"] * 100).astype( int )

# Готовим графику: имеем распределение по группам - годы, используем столбчатую диграмму
fig = px.bar(
    data_frame = df,
    
    x = "Age",   # По оси x отпечатаются возрасты клинета 
    y = "Отток", #"Exited", #"Отток", # По оси y отпечатается количество лояльных и ушедших клиентов
    #text_auto = True, # Отображать надписи на колонках гистограммы
    title = 'Распределение оттока клиентов по годам', #заголовок
    width = 2000,    # Ширина графика
    height = 500,    # Высота графика
    text = "Отток",  # Печатаем величину оттока
    color = "Отток", # Раскрасим по величине оттока
)
fig.update_layout(
    bargap = 0.1, # Ширина вертикальной разделительной вертикальной полосы
    xaxis_title = "Возраст, лет", # Подпись оси x
    yaxis_title = "Доля, %", # Подпись оси y
)
fig.show() # Отображаем

Из графика видим, что
1) До 35 лет доля отток клиентов минимальный (по отношению к притоку), затем он плавно растет
2) К 50 годам сравнивается с притоком и начинает его превосходить. Следует обратить внимание на эту группу
3) с 60 годов отток спадает быстрее, чем приток
4) В 84 года аномалия на графике: единственный год, в котором наблюдался отток с 74 года

9.5. Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [ ]:
# Выбираем нужные признаки
tmp_df = churn_df[['CreditScore','EstimatedSalary','Exited']].copy()

fig = px.scatter(tmp_df, x = "EstimatedSalary", y = "CreditScore", color = "Exited")  
fig.update_layout(
    title = "Зависимость опыта работы от возраста",
    title_x = 0.5,
    yaxis_title = "Кредитный рейтинг", # Подпись оси y
    xaxis_title = "Заработная плата, руб", # Подпись оси x
    
    legend = dict(x = 0, y = 0, xanchor = "center", orientation = "h"),
)

fig.show() # Отображаем


Из графика видно, что кредитный рейтинг слабо зависит от заработной платы вне зависимости от статуса лояльности клиента

9.6. Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

In [177]:
tmp_df = churn_df[['Exited',"Gender"]].copy()

# Пычисляем процентные доли по группам
df = (churn_df.pivot_table(
    index = "Gender",
    values = "Exited",
    aggfunc = 'mean'
) * 100).round().astype( int )
df["NotExited"] = 100 - df["Exited"]
df = df.rename( columns = {"Exited": "Доля ушедших", "NotExited": "Доля лояльных",  }) # Отразятся в легенде

# Готовим графику: имеем распределение по категориям, используем столбчатую диграмму
fig = px.bar( 
    data_frame = df, 
    x = df.index,   # Ось x - Пол
    y = df.columns, # Ось y - Проценты от общего количества в группе
    barmode = 'stack', # Вертикальное расположени е групп
    title = 'Зависимость оттока от наличия пола клиента',
    width = 500,  # Ширина графика
    height = 400, # Высота графика
    color_discrete_map = { "Доля ушедших": "red", "Доля лояльных": "blue" },
    text_auto = True,
    labels = {'variable': 'Статус клиента: '} # Подпись на легенде
)  
fig.update_layout(
    xaxis_title='Пол',
    yaxis_title='% от общего количества клиентов',
    legend = dict( orientation = "h", x = 0, y = 150, title_font_family = "Sitka Small" ) # Легенду расположим сверку слева
)
fig.show()

Видно, что мужчины уходят реже женщин

9.7. Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [178]:
tmp_df = churn_df[["NumOfProducts", 'Exited',"CustomerId"]].copy()
df = tmp_df.pivot_table(
    index = 'NumOfProducts',
    columns = 'Exited',
    values = 'CustomerId',
    fill_value = 0,
    aggfunc = 'count'    
)

df = df.rename( columns = {1: "Ушедшие", 0: "Лояльные"})
df["Доля ушедших"] = (df["Ушедшие"] * 100 / (df["Лояльные"] + df["Ушедшие"])).round().astype( int )
df["Доля лояльных"] = 100 - df["Доля ушедших"] 

# Готовим графику: имеем распределение по группам - годы, используем столбчатую диграмму
fig = px.bar( 
    data_frame = df, 
    x = df.index, # Ось x - Количество услуг
    y = ['Доля лояльных', 'Доля ушедших'],  # Ось y - Процентная доля в группе
    barmode ='group', # Группы расположены по горизонтали
    title = 'Зависимость оттока клиентов от количества услуг',
    labels = {'variable': 'Статус клиента: '}, # Подпись на легенде
    width = 500,  # Ширина графика
    height = 400, # Высота графика
    text_auto = True,
)  
fig.update_layout(
    xaxis_title = 'Количество услуг',
    yaxis_title = '% от общего количества клиентов',
    legend = dict(orientation = "h", x = 0, y = 150, title_font_family = "Sitka Small" ) # Легенду расположим сверку слева
)
fig.show()

Минимум оттока клиентов наблюдлается при 2х услугах у клиента. 

9.8. Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [179]:
tmp_df = churn_df[['Exited',"IsActiveMember","CustomerId"]].copy()
df = tmp_df.pivot_table(
    index = 'IsActiveMember',
    columns='Exited',
    values = 'CustomerId',
    fill_value = 0,
    aggfunc = 'count'    
)
df = df.rename( columns = {1: "Ушедшие", 0: "Лояльные"})
# Вычислим процентные доли
df["Доля ушедших"] = (df["Ушедшие"] * 100 / (df["Лояльные"] + df["Ушедшие"])).round().astype( int )
df["Доля лояльных"] = 100 - df["Доля ушедших"] 

# Готовим графику: имеем распределение по группам - годы, используем столбчатую диграмму
fig = px.bar( 
    data_frame = df, 
    x = df.index, # По оси x - Статус активного клиента
    y = ['Доля лояльных', 'Доля ушедших'], 
    #text = ['Доля лояльных', 'Доля ушедших'], #текст на столбцах
    barmode ='group',
    title = 'Зависимость оттока от наличия статуса активного клиента',
    width = 600,  # Ширина графика
    height = 400, # Высота графика
    text_auto = True, # Подписи величин
    labels = {'variable': 'Статус клиента: '}, # Подпись на легенде
)  
fig.update_layout(
    xaxis_title='Статус активного клиента',
    yaxis_title='% от общего количества клиентов',
    legend = dict(orientation = "h", x = 0, y = 150, title_font_family = "Sitka Small" ) # Легенду расположим сверку слева
)
fig.show()

Видно, что наличие статуса активного клиента снижает желание покануть банк

9.9. В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

In [180]:
tmp_df = churn_df[['Exited',"Geography","CustomerId"]].copy()

df = tmp_df.pivot_table(
    index = 'Geography',
    columns='Exited',
    values = 'CustomerId',
    fill_value = 0,
    aggfunc = 'count'    
)
df = df.rename( columns = {1: "Ушедшие", 0: "Лояльные"})
# Вычисляем процентиные доли
df["Доля ушедших"] = (df["Ушедшие"] * 100 / (df["Лояльные"] + df["Ушедшие"])).round().astype( int )

# Готовим графику: имеем распределение по группам - странам, используем тепловую географичесчкую карту
fig = px.choropleth( 
    data_frame = df, 
    locationmode = "country names", # Режим поиска - пос названиям стран
    locations = list( set( df.index ) ), # Список стран
    scope = 'europe', 
    color = "Доля ушедших", # Поле, управляющее цветом
    range_color = [0, 100], # Диапазон цвета 
    title = 'Доля ущедших клиентов по странам, %', # Заголовок
    width = 800, # Ширина
    height = 500, # Высота
    color_continuous_scale = 'Reds' #палитра цветов       
)  
fig.show()  # Печатаем

Видим, что это Германия. Возможно, это связано с разделением страны на 2 части, требуется дополнительное исследование этой особенности

9.10. Переведите числовой признак CreditScore в категориальный. Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

In [181]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"

Постройте сводную таблицу, строками которой являются категории кредитного рейтинга (CreditScoreCat), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (Tenure). В ячейках сводной таблицы должно находиться среднее по признаку оттока (Exited) — доля ушедших пользователей.

На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [182]:
tmp_df = churn_df.copy()

# Создаем категориальный признак
tmp_df["CreditScoreCat"] = tmp_df["CreditScore"].apply( lambda x: get_credit_score_cat( x ) ).astype( "category" )
# Группируем по 2м параметрам
df = tmp_df.pivot_table(
    index = 'CreditScoreCat',
    columns ='Tenure',
    values = 'Exited',
    fill_value = 0,
    aggfunc = 'mean',
    observed = True
).round(2)

# Готовим графику: имеем отображение декартового произведения категорий в число, используем тепловую карту
fig = px.imshow( df.values,
    title = "Зависимость категории кредитного рейтинга клиента от количества лет пользования услугами банка",
    aspect = "auto",
    labels = dict( 
        x = "Кол-во лет, в течение которых клиент пользуется услугами банка", 
        y = "Категория кредитного рейтинга клиента", 
        color = "Доля"
    ),
    y = df.index.to_list(), # Ось y - Категории
    text_auto = True, # Включает отображение значений в ячейках
    color_continuous_scale = 'ylorbr', # Цветовая шкала
    width = 1000, # Ширина
    height = 600  # Высота
)
fig.show() # Отображаем